In [ ]:
#dependencies
import pandas as pd 
import numpy as np 
import time
import requests
from config import weather_api_key
from datetime import datetime
from citipy import citipy

In [ ]:
# create a set of random latitude and longitude combinations
lats = np.random.uniform(low=-90.000, high=90.000,size=1500)
lngs = np.random.uniform(low=-180.000, high=180.000,size=1500)
lats_lngs = zip(lats, lngs)

In [ ]:
# add latitudes and longitudes to a list
coordinates = list(lats_lngs)

In [ ]:
#create a list for holding the cities
cities = []
#identify nearest city for each lat, lng combo
for coordinate in coordinates:
    city = citipy.nearest_city(coordinate[0], coordinate[1]).city_name

    #if the city is unique then it will be added to cities list
    if city not in cities:
        cities.append(city)
len(cities)

In [ ]:
# Starting URL for Weather Map API Call.
url = "http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=" + weather_api_key

In [ ]:
#create an empty list to hold weather data
city_data = []
#create counters
record_count = 1
set_count = 1

#loop through list of cities and build city url
for i, city in enumerate(cities):
    #group cities in sets of 50 for tracking 
    if(i % 50 == 0 and i >= 50):
        set_count += 1
        record_count = 1
    #create city url
    city_url = url + "&q=" + city.replace(" ","+")
    # add 1 to the record count
    record_count += 1
    
    #run api request for each city
    try:
        #parse json and retrieve data
        city_weather = requests.get(city_url).json()
        #parse needed data
        city_lat = city_weather["coord"]["lat"]
        city_lng = city_weather["coord"]["lon"]
        city_max_temp = city_weather["main"]["temp_max"]
        city_humidity = city_weather["main"]["humidity"]
        city_clouds= city_weather["clouds"]["all"]
        city_wind = city_weather["wind"]["speed"]
        city_country = city_weather["sys"]["country"]
        city_weather_description = city_weather["weather"][0]["description"]
        #append city info into city_data list
        city_data.append({"City": city.title(),
                          "Lat": city_lat,
                          "Lng": city_lng,
                          "Max Temp": city_max_temp,
                          "Humidity": city_humidity,
                          "Cloudiness": city_clouds,
                          "Wind Speed": city_wind,
                          "Country": city_country,
                          "Current Description": city_weather_description,
                          "Rain (inches)": city_rain,
                          "Snow (inches)": city_snow})
    #if an error is encountered skip the city
    except:
        pass
    try:
        city_rain = city_weather["rain"]["1h"]
    except:
        city_rain = 0
        pass
    try:
        city_snow = city_weather["snow"]["1h"]
    except:
        city_snow = 0
        pass

In [ ]:
#create dataframe from list of dictionaries
city_data_df = pd.DataFrame(city_data)

In [ ]:
#re-order column titles 
new_column_order = ["City", "Country", "Lat", "Lng", "Max Temp", "Humidity", "Cloudiness", "Wind Speed", "Current Description", "Rain (inches)","Snow (inches)"]
city_data_df = city_data_df[new_column_order]

In [56]:
#create output file (csv)
output_data_file = "weather_data/weather_database.csv"
#export city_data into a csv
city_data_df.to_csv(output_data_file, index_label="City_ID")